In [1]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = (20, 10)

In [2]:
aapl = yf.download("META", start="2015-01-01", end="2020-01-01").reset_index()

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
def get_stoch_osc(high, low, close, k_lookback, d_lookback):
    lowest_low = low.rolling(k_lookback).min()
    highest_high = high.rolling(k_lookback).max()
    k_line = ((close - lowest_low) / (highest_high - lowest_low)) * 100
    d_line = k_line.rolling(d_lookback).mean()
    return k_line, d_line


aapl["%k"], aapl["%d"] = get_stoch_osc(aapl["High"], aapl["Low"], aapl["Close"], 14, 3)
aapl.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,%k,%d
1253,2019-12-24,206.300003,206.789993,205.000000,205.119995,204.902588,6046300,77.496747,84.056754
1254,2019-12-26,205.570007,207.820007,205.309998,207.789993,207.569763,9350700,94.811912,85.559860
1255,2019-12-27,208.669998,208.929993,206.589996,208.100006,207.879440,10284200,94.733586,89.014082
1256,2019-12-30,207.860001,207.899994,203.899994,204.410004,204.193344,10524300,71.319857,86.955118
1257,2019-12-31,204.000000,205.559998,203.600006,205.250000,205.032455,8953500,76.649785,80.901076


In [4]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span=fast, adjust=False).mean()
    exp2 = price.ewm(span=slow, adjust=False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns={"Close": "macd"})
    signal = pd.DataFrame(macd.ewm(span=smooth, adjust=False).mean()).rename(
        columns={"macd": "signal"}
    )
    hist = pd.DataFrame(macd["macd"] - signal["signal"]).rename(columns={0: "hist"})
    return macd, signal, hist


aapl["macd"] = get_macd(aapl["Close"], 26, 12, 9)[0]
aapl["macd_signal"] = get_macd(aapl["Close"], 26, 12, 9)[1]
aapl["macd_hist"] = get_macd(aapl["Close"], 26, 12, 9)[2]
aapl = aapl.dropna()
aapl.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,%k,%d,macd,macd_signal,macd_hist
1253,2019-12-24,206.300003,206.789993,205.000000,205.119995,204.902588,6046300,77.496747,84.056754,2.595922,2.212383,0.383539
1254,2019-12-26,205.570007,207.820007,205.309998,207.789993,207.569763,9350700,94.811912,85.559860,2.832828,2.336472,0.496356
1255,2019-12-27,208.669998,208.929993,206.589996,208.100006,207.879440,10284200,94.733586,89.014082,3.010885,2.471355,0.539531
1256,2019-12-30,207.860001,207.899994,203.899994,204.410004,204.193344,10524300,71.319857,86.955118,2.821718,2.541427,0.280290
1257,2019-12-31,204.000000,205.559998,203.600006,205.250000,205.032455,8953500,76.649785,80.901076,2.708362,2.574814,0.133547


In [5]:
def implement_stoch_macd_strategy(prices, k, d, macd, macd_signal):
    buy_price = []
    sell_price = []
    stoch_macd_signal = []
    signal = 0

    for i in range(len(prices)):
        if k[i] < 30 and d[i] < 30 and macd[i] < -2 and macd_signal[i] < -2:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)

        elif k[i] > 70 and d[i] > 70 and macd[i] > 2 and macd_signal[i] > 2:
            if signal != -1 and signal != 0:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)

        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_macd_signal.append(0)

    return buy_price, sell_price, stoch_macd_signal


aapl = aapl.reset_index(drop=True)
buy_price, sell_price, stoch_macd_signal = implement_stoch_macd_strategy(
    aapl["Close"], aapl["%k"], aapl["%d"], aapl["macd"], aapl["macd_signal"]
)

In [6]:
position = []
for i in range(len(stoch_macd_signal)):
    if stoch_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(aapl["Close"])):
    if stoch_macd_signal[i] == 1:
        position[i] = 1
    elif stoch_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i - 1]

close_price = aapl["Close"]
k_line = aapl["%k"]
d_line = aapl["%d"]
macd_line = aapl["macd"]
signal_line = aapl["macd_signal"]
stoch_macd_signal = (
    pd.DataFrame(stoch_macd_signal)
    .rename(columns={0: "stoch_macd_signal"})
    .set_index(aapl.index)
)
position = (
    pd.DataFrame(position)
    .rename(columns={0: "stoch_macd_position"})
    .set_index(aapl.index)
)

frames = [
    close_price,
    k_line,
    d_line,
    macd_line,
    signal_line,
    stoch_macd_signal,
    position,
]
strategy = pd.concat(frames, join="inner", axis=1)

strategy

,Close,%k,%d,macd,macd_signal,stoch_macd_signal,stoch_macd_position
0,77.500000,77.952710,75.021031,-0.305116,-0.465654,0,1
1,75.779999,44.094430,65.726172,-0.376416,-0.447806,0,1
2,76.239998,53.149527,58.398889,-0.391293,-0.436503,0,1
3,78.000000,87.795226,61.679728,-0.258090,-0.400821,0,1
4,75.910004,48.073075,63.005943,-0.317511,-0.384159,0,1
...,...,...,...,...,...,...,...
1238,205.119995,77.496747,84.056754,2.595922,2.212383,0,0
1239,207.789993,94.811912,85.559860,2.832828,2.336472,0,0
1240,208.100006,94.733586,89.014082,3.010885,2.471355,0,0
1241,204.410004,71.319857,86.955118,2.821718,2.541427,0,0


In [7]:
aapl_ret = pd.DataFrame(np.diff(aapl["Close"])).rename(columns={0: "returns"})
stoch_macd_strategy_ret = []

for i in range(len(aapl_ret)):
    try:
        returns = aapl_ret["returns"][i] * strategy["stoch_macd_position"][i]
        stoch_macd_strategy_ret.append(returns)
    except:
        pass

stoch_macd_strategy_ret_df = pd.DataFrame(stoch_macd_strategy_ret).rename(
    columns={0: "stoch_macd_returns"}
)

investment_value = 100000
number_of_stocks = floor(investment_value / aapl["Close"][0])
stoch_macd_investment_ret = []

for i in range(len(stoch_macd_strategy_ret_df["stoch_macd_returns"])):
    returns = number_of_stocks * stoch_macd_strategy_ret_df["stoch_macd_returns"][i]
    stoch_macd_investment_ret.append(returns)

stoch_macd_investment_ret_df = pd.DataFrame(stoch_macd_investment_ret).rename(
    columns={0: "investment_returns"}
)
total_investment_ret = round(sum(stoch_macd_investment_ret_df["investment_returns"]), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(
    cl(
        "Profit gained from the STOCH MACD strategy by investing $100k in AAPL : {}".format(
            total_investment_ret
        ),
        attrs=["bold"],
    )
)
print(
    cl(
        "Profit percentage of the STOCH MACD strategy : {}%".format(profit_percentage),
        attrs=["bold"],
    )
)

Profit gained from the STOCH MACD strategy by investing $100k in AAPL : 104593.2
Profit percentage of the STOCH MACD strategy : 104%
